In [1]:
import edge_tts
import asyncio
import util    
import json
import sys
from IPython.display import Audio
from speakers.processors import load_preprocess,RvcProcessorData
from speakers.common.utils import get_abs_path
from omegaconf import OmegaConf

2023-08-18 13:40:21 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2023-08-18 13:40:22 | INFO | faiss.loader | Loading faiss with AVX2 support.
2023-08-18 13:40:22 | INFO | faiss.loader | Successfully loaded faiss with AVX2 support.
/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/pydantic/_internal/_fields.py:127: UserWarning: Field "model_index" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/media/gpt4-pdf-chatbot-langchain/pyenv-rvc-speakers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


from speakers.common.registry import registry rvc_speakers
from speakers.common.registry import registry vits_to_voice
from speakers.common.registry import registry voice_task
 Key with which the item will be registered {'device': 'cuda:0'}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60}
 Key with which the item will be registered {'device': 'cuda:0', 'is_half': True, 'x_pad': 3, 'x_query': 10, 'x_center': 60, 'x_max': 65}


In [2]:
## 初始化 处理器

speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)
load_preprocess(config=config.get('preprocess'))
 

{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}], 'tasks': [{'voice_task': {'name': 'voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}]}
{'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}


2023-08-18 13:40:24 | INFO | root | Loaded checkpoint '/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/vits/model/G_953000.pth' (iteration 630)
2023-08-18 13:40:24 | INFO | fairseq.tasks.hubert_pretraining | current directory is /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/notebook
2023-08-18 13:40:24 | INFO | fairseq.tasks.hubert_pretraining | HubertPretrainingTask Config {'_name': 'hubert_pretraining', 'data': 'metadata', 'fine_tuning': False, 'labels': ['km'], 'label_dir': 'label', 'label_rate': 50.0, 'sample_rate': 16000, 'normalize': False, 'enable_padding': False, 'max_keep_size': None, 'max_sample_size': 250000, 'min_sample_size': 32000, 'single_target': False, 'random_crop': True, 'pad_audio': False}
2023-08-18 13:40:24 | INFO | fairseq.models.hubert.hubert | HubertModel Config: {'_name': 'hubert', 'label_rate': 50.0, 'extractor_mode': default, 'encoder_layers': 12, 'encoder_embed_dim': 768, 'encoder_ffn_embed_dim': 3072, 'encoder_attention_heads': 12, 'activation_fn': gelu, '

Models loaded
{'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}
/media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/rmvpe.pt
Loading model: yiqing
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/yiqing/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: bob
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/bob/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: arianagrande
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/arianagrande/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys matched successfully>
Loading model: syz
Loading model model_info_config_file: /media/gpt4-pdf-chatbot-langchain/RVC-Speakers/rvc/model/syz/config.json
gin_channels: 256 self.spk_embed_dim: 109
<All keys m

In [3]:
from speakers.tasks import get_task,load_task
 

# 加载任务实例
speakers_config_file='speakers.yaml'
config = OmegaConf.load(get_abs_path(speakers_config_file))
print(config)

load_task(config.get("tasks"))

{'preprocess': [{'vits_processor': {'name': 'vits_to_voice', 'vits_model_path': 'model/G_953000.pth', 'voice_config_file': 'model/config.json'}}, {'rvc_processor': {'name': 'rvc_speakers', 'hubert_model_path': 'model/hubert_base.pt', 'rvc_config_file': 'rvc.yaml'}}], 'tasks': [{'voice_task': {'name': 'voice_task', 'preprocess': [{'vits': {'processor': 'vits_processor', 'processor_name': 'VITS'}}, {'rvc': {'processor': 'rvc_processor', 'processor_name': 'RVC'}}]}}]}


In [4]:
voice_task = get_task("voice_task")

In [12]:
from speakers.processors import get_processors,VitsProcessorData
from speakers.tasks import VoiceFlowData, Runner

from collections import deque

vits_voice = get_processors('vits_processor')

rvc_processors = get_processors('rvc_processor')

# 创建双向链表
linked_list = deque()
linked_list.append('VITS')
linked_list.append('RVC') 


#noise_scale_w: noise_scale_w(控制音素发音长度)
noise_scale_w=1
#noise_scale(控制感情变化程度)
noise_scale = 0.5
#length_scale(控制整体语速)
speed = 1

# 创建一个 VitsProcessorData 实例
vits_processor_data = VitsProcessorData(
    text='我是你的情感导师，青春的时间转瞬即逝，秋水于海又岂有一只浮游',
    language=vits_voice.lang.index('简体中文'),
    speaker_id=0,
    noise_scale=noise_scale,
    speed=speed,
    noise_scale_w=noise_scale_w
)



loaded_models=[
        '%s - %s' % (
            m['metadata'].get('source', 'Unknown'),
            m['metadata'].get('name')
        )
        for m in rvc_processors.loaded_models
]
 


#变调(整数, 半音数量, 升八度12降八度-12)
f0_up_key=-1
f0_method =["pm", "harvest", "crepe", "rmvpe"]

#检索特征占比
index_rate = 0.9
#>=3则使用对harvest音高识别的结果使用中值滤波，数值为滤波半径，使用可以削弱哑音
filter_radius=1
#输入源音量包络替换输出音量包络融合比例，越靠近1越使用输出包络
rms_mix_rate=1
#后处理重采样至最终采样率，0为不进行重采样
resample_rate=0

rvc_processor_data = RvcProcessorData(  
    model_index= 0,
    f0_up_key= f0_up_key,
    f0_method= f0_method[3],
    index_rate= index_rate,
    filter_radius= filter_radius,
    rms_mix_rate= rms_mix_rate,
    resample_sr= resample_rate 
)


# 创建一个 VoiceFlowData 实例，并将 VitsProcessorData 实例作为参数传递
voice_flow_data = VoiceFlowData(vits=vits_processor_data,
                               rvc=rvc_processor_data)

# 创建 Runner 实例并传递上面创建的 VoiceFlowData 实例作为参数
runner = Runner(
    task_id="1",
    flow_data=voice_flow_data,
    processor_q=linked_list
)
 



In [13]:
preprocess_object = voice_task.preprocess_dict.get(vits_processor_data.type)
preprocess_object.match


<bound method VitsToVoice.match of <speakers.processors.vits_to_voice.VitsToVoice object at 0x7f8c54bf9390>>

In [14]:

voice_task.prepare(runner=runner)
rate,output_audio = await voice_task.dispatch(runner=runner)


2023-08-18 13:42:24 | INFO | base_task_runner | dispatch
2023-08-18 13:42:24 | INFO | base_task_runner | Running voice_task


-1
loading rmvpe model
npy: 0.026128530502319336s, f0: 0.6579000949859619s, infer: 0.13386845588684082s


In [15]:
from scipy.io.wavfile import write as write_wav
from IPython.display import Audio
print(rate)
# save audio to disk
write_wav(f"test.wav", rate, output_audio) 

# play text in notebook
Audio(output_audio, rate=rate)

48000
